# SwellSight Real-to-Synthetic Pipeline - Depth-Anything-V2 Depth Extraction (Enhanced)

This notebook implements the enhanced Depth-Anything-V2 depth estimation phase with advanced error handling and memory management.

## Overview
This enhanced notebook provides:
- Depth-Anything-V2 model initialization with memory optimization
- Advanced error handling with GPU fallback mechanisms
- Individual image error handling with batch continuation
- Comprehensive depth quality assessment
- Adaptive memory management and cleanup
- Detailed error reporting and recovery instructions

## Key Improvements
- **Memory Management**: Dynamic memory monitoring and adaptive processing
- **Error Recovery**: GPU memory error handling with CPU fallback
- **Quality Assessment**: Enhanced depth map quality validation
- **Batch Resilience**: Individual image failures don't stop batch processing
- **Performance Monitoring**: Real-time memory and performance tracking

---

In [ ]:
import sys
import os
import gc
import psutil
from pathlib import Path
import json
import logging
from datetime import datetime
import time
from collections import defaultdict
from dataclasses import dataclass
from typing import Dict, List, Optional, Tuple

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
from tqdm.auto import tqdm
import torch
import torch.nn.functional as F
from transformers import pipeline
import cv2
import warnings
warnings.filterwarnings('ignore')

# Add utils to path for shared utilities
sys.path.append(str(Path.cwd() / 'utils'))

# Import shared utilities
from config_manager import load_config
from data_flow_manager import load_previous_results, save_stage_results
from error_handler import retry_with_backoff, handle_gpu_memory_error
from memory_optimizer import get_optimal_batch_size, cleanup_variables
from progress_tracker import create_progress_bar
from data_validator import validate_image_quality

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

print("🚀 Enhanced Depth-Anything-V2 Extraction Pipeline")
print("   Advanced error handling and memory management enabled")

In [ ]:
@dataclass
class DepthResult:
    """Enhanced data class to store depth extraction results."""
    depth_map: np.ndarray
    depth_quality_score: float
    processing_time: float
    model_used: str
    memory_usage: Dict[str, float]
    image_size: Tuple[int, int]
    preprocessing_applied: List[str]

@dataclass
class ProcessingStats:
    """Statistics for batch processing."""
    total_processed: int = 0
    successful: int = 0
    failed: int = 0
    gpu_fallbacks: int = 0
    memory_cleanups: int = 0
    total_time: float = 0.0
    
class EnhancedDepthAnythingV2Extractor:
    """Enhanced Depth-Anything-V2 extractor with advanced error handling and memory management."""

    def __init__(self, model_name="depth-anything/Depth-Anything-V2-Large", device="cuda", storage_path=None):
        self.device = device
        self.storage_path = Path(storage_path) if storage_path else None
        self.model_name = model_name
        self.pipe = None
        
        # Memory management settings
        self.memory_threshold = 0.85  # 85% memory usage threshold
        self.critical_memory_threshold = 0.95  # 95% critical threshold
        self.consecutive_failures = 0
        self.max_consecutive_failures = 5
        self.memory_cleanup_interval = 10  # Clean memory every N images
        
        # Error tracking
        self.error_history = defaultdict(list)
        self.recovery_strategies = {
            'OutOfMemoryError': self._handle_oom_error,
            'RuntimeError': self._handle_runtime_error,
            'FileNotFoundError': self._handle_file_error,
            'ValueError': self._handle_value_error
        }
        
        print(f"   Loading Enhanced Depth-Anything-V2 model: {model_name} on {device}...")
        self._initialize_model()

    def _initialize_model(self):
        """Initialize model with comprehensive error handling."""
        try:
            # Clear any existing GPU memory before loading
            if self.device == "cuda" and torch.cuda.is_available():
                torch.cuda.empty_cache()
                initial_memory = self._get_memory_info()
                print(f"   Initial GPU memory: {initial_memory['allocated_gb']:.2f}/{initial_memory['total_gb']:.2f} GB")
            
            # Initialize the depth estimation pipeline with memory optimization
            self.pipe = pipeline(
                task="depth-estimation",
                model=self.model_name,
                device=0 if self.device == "cuda" else -1,
                torch_dtype=torch.float16 if self.device == "cuda" else torch.float32
            )
            
            if self.device == "cuda" and torch.cuda.is_available():
                post_load_memory = self._get_memory_info()
                model_memory = post_load_memory['allocated_gb'] - (initial_memory['allocated_gb'] if 'initial_memory' in locals() else 0)
                print(f"   Post-load GPU memory: {post_load_memory['allocated_gb']:.2f} GB")
                print(f"   Model memory footprint: {model_memory:.2f} GB")
            
            print(f"   ✅ Model loaded successfully: {self.model_name}")
            
        except Exception as e:
            print(f"❌ Error loading model: {e}")
            self._log_error('ModelInitializationError', str(e))
            raise

    def _get_memory_info(self) -> Dict[str, float]:
        """Get comprehensive memory usage information."""
        info = {'usage_ratio': 0.0, 'is_high': False, 'is_critical': False}
        
        if self.device == "cuda" and torch.cuda.is_available():
            allocated = torch.cuda.memory_allocated() / 1e9
            reserved = torch.cuda.memory_reserved() / 1e9
            total = torch.cuda.get_device_properties(0).total_memory / 1e9
            usage_ratio = allocated / total
            
            info.update({
                'allocated_gb': allocated,
                'reserved_gb': reserved,
                'total_gb': total,
                'usage_ratio': usage_ratio,
                'is_high': usage_ratio > self.memory_threshold,
                'is_critical': usage_ratio > self.critical_memory_threshold
            })
        
        # Add system memory info
        system_memory = psutil.virtual_memory()
        info.update({
            'system_memory_gb': system_memory.total / 1e9,
            'system_memory_used_gb': system_memory.used / 1e9,
            'system_memory_percent': system_memory.percent
        })
        
        return info

    def _handle_memory_pressure(self, force_cleanup=False) -> bool:
        """Handle high memory usage with progressive cleanup strategies."""
        memory_info = self._get_memory_info()
        
        if not memory_info['is_high'] and not force_cleanup:
            return True
        
        logger.info(f"Memory pressure detected: {memory_info['usage_ratio']:.1%}")
        
        # Progressive cleanup strategies
        cleanup_strategies = [
            ("torch_cache", lambda: torch.cuda.empty_cache() if torch.cuda.is_available() else None),
            ("torch_sync", lambda: torch.cuda.synchronize() if torch.cuda.is_available() else None),
            ("garbage_collect", lambda: gc.collect()),
            ("force_gc", lambda: [gc.collect() for _ in range(3)])  # Multiple GC passes
        ]
        
        for strategy_name, cleanup_func in cleanup_strategies:
            try:
                cleanup_func()
                new_memory_info = self._get_memory_info()
                
                improvement = memory_info['usage_ratio'] - new_memory_info['usage_ratio']
                logger.info(f"Applied {strategy_name}: {improvement:.1%} memory freed")
                
                if not new_memory_info['is_critical']:
                    return True
                    
                memory_info = new_memory_info
                
            except Exception as e:
                logger.warning(f"Cleanup strategy {strategy_name} failed: {e}")
        
        # If still critical, recommend CPU fallback
        final_memory_info = self._get_memory_info()
        if final_memory_info['is_critical']:
            logger.error(f"Memory usage still critical after cleanup: {final_memory_info['usage_ratio']:.1%}")
            return False
        
        return True

    def _adaptive_preprocessing(self, image: Image.Image) -> Tuple[Image.Image, List[str]]:
        """Adaptively preprocess image based on memory constraints and image characteristics."""
        preprocessing_applied = []
        memory_info = self._get_memory_info()
        original_size = image.size
        
        # Memory-based resizing
        if memory_info['is_high']:
            scale_factor = 0.75 if memory_info['is_critical'] else 0.85
            new_size = (int(original_size[0] * scale_factor), int(original_size[1] * scale_factor))
            image = image.resize(new_size, Image.Resampling.LANCZOS)
            preprocessing_applied.append(f"memory_resize_{scale_factor}")
            logger.info(f"Applied memory-based resize: {original_size} -> {new_size}")
        
        # Size-based optimization
        max_dimension = max(image.size)
        if max_dimension > 2048:  # Very large images
            scale_factor = 2048 / max_dimension
            new_size = (int(image.size[0] * scale_factor), int(image.size[1] * scale_factor))
            image = image.resize(new_size, Image.Resampling.LANCZOS)
            preprocessing_applied.append(f"size_optimize_{scale_factor:.2f}")
            logger.info(f"Applied size optimization: {original_size} -> {new_size}")
        
        return image, preprocessing_applied

    def _calculate_enhanced_quality_score(self, depth_map: np.ndarray) -> Dict[str, float]:
        """Calculate comprehensive depth map quality metrics."""
        try:
            # Normalize depth map
            depth_norm = (depth_map - depth_map.min()) / (depth_map.max() - depth_map.min() + 1e-8)
            
            # Multiple quality metrics
            metrics = {}
            
            # 1. Standard deviation (depth variation)
            metrics['std_score'] = np.std(depth_norm)
            
            # 2. Gradient magnitude (edge information)
            grad_x = np.gradient(depth_norm, axis=1)
            grad_y = np.gradient(depth_norm, axis=0)
            gradient_magnitude = np.sqrt(grad_x**2 + grad_y**2)
            metrics['gradient_score'] = np.mean(gradient_magnitude)
            
            # 3. Entropy (information content)
            hist, _ = np.histogram(depth_norm, bins=256, range=(0, 1))
            hist = hist / hist.sum()
            hist = hist[hist > 0]  # Remove zero bins
            metrics['entropy_score'] = -np.sum(hist * np.log2(hist)) / 8.0  # Normalize by max entropy
            
            # 4. Dynamic range
            metrics['dynamic_range'] = depth_map.max() - depth_map.min()
            
            # 5. Smoothness (inverse of total variation)
            tv = np.sum(np.abs(grad_x)) + np.sum(np.abs(grad_y))
            metrics['smoothness_score'] = 1.0 / (1.0 + tv / depth_map.size)
            
            # Combined quality score (weighted average)
            weights = {
                'std_score': 0.3,
                'gradient_score': 0.25,
                'entropy_score': 0.25,
                'smoothness_score': 0.2
            }
            
            combined_score = sum(weights[key] * metrics[key] for key in weights.keys())
            metrics['combined_score'] = min(1.0, max(0.0, combined_score))
            
            return metrics
            
        except Exception as e:
            logger.warning(f"Enhanced quality calculation failed: {e}")
            return {
                'combined_score': 0.5,
                'std_score': 0.5,
                'gradient_score': 0.5,
                'entropy_score': 0.5,
                'smoothness_score': 0.5,
                'dynamic_range': 0.0
            }

    def _log_error(self, error_type: str, error_message: str, image_path: str = None):
        """Log error with context for analysis."""
        error_entry = {
            'timestamp': datetime.now().isoformat(),
            'error_type': error_type,
            'message': error_message,
            'image_path': str(image_path) if image_path else None,
            'memory_info': self._get_memory_info(),
            'consecutive_failures': self.consecutive_failures
        }
        self.error_history[error_type].append(error_entry)

    def _handle_oom_error(self, image_path: str) -> Optional[DepthResult]:
        """Handle Out of Memory errors with progressive strategies."""
        logger.warning(f"OOM error for {image_path}, attempting recovery...")
        
        # Force memory cleanup
        if not self._handle_memory_pressure(force_cleanup=True):
            logger.error("Memory cleanup failed, skipping image")
            return None
        
        try:
            # Try with smaller image
            image = Image.open(image_path).convert('RGB')
            original_size = image.size
            
            # Aggressive resizing for OOM recovery
            scale_factor = 0.5
            new_size = (int(original_size[0] * scale_factor), int(original_size[1] * scale_factor))
            image = image.resize(new_size, Image.Resampling.LANCZOS)
            
            logger.info(f"OOM recovery: resized {original_size} -> {new_size}")
            
            # Try extraction with reduced image
            with torch.no_grad():
                result = self.pipe(image)
                depth_map = np.array(result['depth'])
            
            # Resize depth map back to original dimensions
            depth_map_resized = cv2.resize(depth_map, original_size, interpolation=cv2.INTER_CUBIC)
            
            quality_metrics = self._calculate_enhanced_quality_score(depth_map_resized)
            
            return DepthResult(
                depth_map=depth_map_resized,
                depth_quality_score=quality_metrics['combined_score'],
                processing_time=0.0,  # Not tracking time for recovery
                model_used=f"{self.model_name} (OOM recovery)",
                memory_usage=self._get_memory_info(),
                image_size=original_size,
                preprocessing_applied=[f"oom_recovery_resize_{scale_factor}"]
            )
            
        except Exception as e:
            logger.error(f"OOM recovery failed: {e}")
            return None

    def _handle_runtime_error(self, image_path: str) -> Optional[DepthResult]:
        """Handle runtime errors."""
        logger.warning(f"Runtime error for {image_path}, attempting model reload...")
        
        try:
            # Try reinitializing the pipeline
            self.cleanup()
            self._initialize_model()
            
            # Retry extraction
            return self.extract_depth(image_path, store_result=False)
            
        except Exception as e:
            logger.error(f"Runtime error recovery failed: {e}")
            return None

    def _handle_file_error(self, image_path: str) -> Optional[DepthResult]:
        """Handle file-related errors."""
        logger.error(f"File error: {image_path} not accessible")
        return None

    def _handle_value_error(self, image_path: str) -> Optional[DepthResult]:
        """Handle value errors (e.g., corrupted images)."""
        logger.warning(f"Value error for {image_path}, attempting image repair...")
        
        try:
            # Try to load and re-save the image to fix minor corruption
            image = Image.open(image_path).convert('RGB')
            
            # Basic image validation
            if image.size[0] < 32 or image.size[1] < 32:
                raise ValueError("Image too small")
            
            # Retry extraction
            return self.extract_depth(image_path, store_result=False)
            
        except Exception as e:
            logger.error(f"Image repair failed: {e}")
            return None

    def extract_depth(self, image_path: str, store_result: bool = True) -> DepthResult:
        """Extract depth map with comprehensive error handling and recovery."""
        start_time = datetime.now()
        
        try:
            # Pre-processing memory check
            memory_info = self._get_memory_info()
            if memory_info['is_critical']:
                if not self._handle_memory_pressure():
                    raise RuntimeError("Critical memory usage, cannot process image")
            
            # Load and validate image
            image = Image.open(image_path).convert('RGB')
            
            # Validate image quality
            quality_check = validate_image_quality(str(image_path))
            if not quality_check['is_valid']:
                raise ValueError(f"Image quality validation failed: {quality_check['issues']}")
            
            # Adaptive preprocessing
            processed_image, preprocessing_applied = self._adaptive_preprocessing(image)
            
            # Extract depth using pipeline
            with torch.no_grad():
                result = self.pipe(processed_image)
                depth_map = np.array(result['depth'])
            
            # If image was resized, resize depth map back
            if processed_image.size != image.size:
                depth_map = cv2.resize(depth_map, image.size, interpolation=cv2.INTER_CUBIC)
                preprocessing_applied.append("depth_resize_back")
            
            # Calculate processing time
            processing_time = (datetime.now() - start_time).total_seconds()
            
            # Calculate enhanced quality metrics
            quality_metrics = self._calculate_enhanced_quality_score(depth_map)
            
            # Store result if requested
            if store_result and self.storage_path:
                self.storage_path.mkdir(parents=True, exist_ok=True)
                save_path = self.storage_path / f"{Path(image_path).stem}_depth.npy"
                np.save(save_path, depth_map)
            
            # Reset consecutive failures on success
            self.consecutive_failures = 0
            
            return DepthResult(
                depth_map=depth_map,
                depth_quality_score=quality_metrics['combined_score'],
                processing_time=processing_time,
                model_used=self.model_name,
                memory_usage=self._get_memory_info(),
                image_size=image.size,
                preprocessing_applied=preprocessing_applied
            )
            
        except Exception as e:
            # Increment consecutive failures
            self.consecutive_failures += 1
            
            # Log error with context
            error_type = type(e).__name__
            self._log_error(error_type, str(e), image_path)
            
            # Try recovery strategies
            if error_type in self.recovery_strategies and self.consecutive_failures <= self.max_consecutive_failures:
                logger.info(f"Attempting recovery for {error_type}...")
                recovery_result = self.recovery_strategies[error_type](image_path)
                if recovery_result:
                    logger.info(f"Recovery successful for {image_path}")
                    return recovery_result
            
            # If recovery failed or too many consecutive failures, re-raise
            logger.error(f"Failed to process {image_path} after recovery attempts: {e}")
            raise e

    def get_error_summary(self) -> Dict:
        """Get comprehensive error summary for reporting."""
        summary = {
            'total_error_types': len(self.error_history),
            'consecutive_failures': self.consecutive_failures,
            'error_breakdown': {}
        }
        
        for error_type, errors in self.error_history.items():
            summary['error_breakdown'][error_type] = {
                'count': len(errors),
                'latest': errors[-1]['timestamp'] if errors else None,
                'sample_message': errors[-1]['message'] if errors else None
            }
        
        return summary

    def cleanup(self):
        """Comprehensive cleanup of model resources."""
        if self.pipe is not None:
            del self.pipe
            self.pipe = None
        
        # Force memory cleanup
        self._handle_memory_pressure(force_cleanup=True)
        
        logger.info("Model cleanup completed")

print("✅ Enhanced DepthAnythingV2Extractor class defined")

In [ ]:
# Load configuration and setup
print("🔄 Loading configuration and processing batch...")

# Check if running in Google Colab
if 'google.colab' in sys.modules:
    from google.colab import drive
    drive.mount('/content/drive')
    PROJECT_PATH = Path('/content/drive/MyDrive/SwellSight')
else:
    PROJECT_PATH = Path.cwd()

try:
    # Load pipeline configuration
    PIPELINE_CONFIG = load_config(PROJECT_PATH / 'config.json')
    
    # Load processing batch from previous stage
    PROCESSING_BATCH = load_previous_results(
        stage_name="data_preprocessing",
        required_files=["processed_images.json", "quality_report.json"]
    )
    
    print("✅ Configuration and batch loaded successfully")
    
except Exception as e:
    print(f"❌ Failed to load configuration or batch: {e}")
    print("Please ensure you have run the previous notebooks and have valid configuration.")
    sys.exit(1)

# Configure device with enhanced detection
print("\n🔧 Enhanced device configuration...")

if torch.cuda.is_available():
    device = "cuda"
    gpu_name = torch.cuda.get_device_name(0)
    gpu_memory = torch.cuda.get_device_properties(0).total_memory / 1e9
    
    print(f"   GPU: {gpu_name}")
    print(f"   Memory: {gpu_memory:.1f} GB")
    
    # Enhanced model selection based on memory
    if gpu_memory < 6:
        selected_model = "depth-anything/Depth-Anything-V2-Small"
        print(f"   ⚠️ Limited memory: Using Small model")
    elif gpu_memory < 12:
        selected_model = "depth-anything/Depth-Anything-V2-Base"
        print(f"   ✅ Adequate memory: Using Base model")
    else:
        selected_model = "depth-anything/Depth-Anything-V2-Large"
        print(f"   🚀 Excellent memory: Using Large model")
else:
    device = "cpu"
    gpu_memory = 0
    selected_model = "depth-anything/Depth-Anything-V2-Small"
    print(f"   ⚠️ CPU mode: Using Small model")

# Set up paths
DEPTH_OUTPUT_PATH = Path(PIPELINE_CONFIG['paths']['output_dir']) / 'depth_maps'
DEPTH_OUTPUT_PATH.mkdir(parents=True, exist_ok=True)

print(f"\n📁 Output path: {DEPTH_OUTPUT_PATH}")

In [ ]:
# Initialize enhanced depth extractor
print("🧠 Initializing Enhanced Depth-Anything-V2 extractor...")

try:
    depth_extractor = EnhancedDepthAnythingV2Extractor(
        model_name=selected_model,
        device=device,
        storage_path=str(DEPTH_OUTPUT_PATH)
    )
    
    print("✅ Enhanced extractor initialized successfully!")
    
except Exception as e:
    print(f"❌ Failed to initialize extractor: {e}")
    
    # Enhanced fallback with multiple strategies
    if device == "cuda":
        print("\n🔄 Attempting enhanced GPU fallback strategies...")
        
        fallback_strategies = [
            ("depth-anything/Depth-Anything-V2-Base", "cuda"),
            ("depth-anything/Depth-Anything-V2-Small", "cuda"),
            ("depth-anything/Depth-Anything-V2-Small", "cpu")
        ]
        
        for model, dev in fallback_strategies:
            try:
                print(f"   Trying {model} on {dev}...")
                depth_extractor = EnhancedDepthAnythingV2Extractor(
                    model_name=model,
                    device=dev,
                    storage_path=str(DEPTH_OUTPUT_PATH)
                )
                print(f"   ✅ Fallback successful: {model} on {dev}")
                selected_model = model
                device = dev
                break
            except Exception as fallback_error:
                print(f"   ❌ Fallback failed: {fallback_error}")
                continue
        else:
            print("❌ All fallback strategies failed")
            raise
    else:
        raise

In [ ]:
# Enhanced batch processing with comprehensive error handling
print("🚀 Starting enhanced batch depth extraction...")

# Get images to process
images_to_process = PROCESSING_BATCH.get('processed_images', [])
total_images = len(images_to_process)

if total_images == 0:
    print("❌ No images found to process")
    sys.exit(1)

print(f"   Total images: {total_images}")
print(f"   Model: {selected_model}")
print(f"   Device: {device}")

# Initialize enhanced tracking
stats = ProcessingStats()
successful_results = []
failed_results = []
processing_times = []
quality_scores = []
memory_usage_history = []

# Enhanced batch processing with progress tracking
start_time = datetime.now()

with create_progress_bar(total_images, "Enhanced depth extraction") as pbar:
    for i, image_info in enumerate(images_to_process):
        stats.total_processed += 1
        
        try:
            # Get image path
            image_path = image_info.get('path') or image_info.get('file_path')
            if not image_path:
                raise ValueError("No image path found")
            
            image_path = Path(image_path)
            if not image_path.exists():
                raise FileNotFoundError(f"Image not found: {image_path}")
            
            # Extract depth with enhanced error handling
            result = depth_extractor.extract_depth(str(image_path), store_result=True)
            
            # Track successful result
            stats.successful += 1
            successful_results.append({
                'image_path': str(image_path),
                'depth_quality_score': result.depth_quality_score,
                'processing_time': result.processing_time,
                'model_used': result.model_used,
                'image_size': result.image_size,
                'preprocessing_applied': result.preprocessing_applied,
                'memory_usage': result.memory_usage
            })
            
            processing_times.append(result.processing_time)
            quality_scores.append(result.depth_quality_score)
            memory_usage_history.append(result.memory_usage)
            
        except Exception as e:
            # Enhanced error handling
            stats.failed += 1
            error_type = type(e).__name__
            
            failed_results.append({
                'image_path': str(image_path),
                'error': str(e),
                'error_type': error_type,
                'timestamp': datetime.now().isoformat()
            })
            
            logger.error(f"Failed to process {image_path}: {e}")
        
        # Enhanced memory management
        if (i + 1) % depth_extractor.memory_cleanup_interval == 0:
            memory_info = depth_extractor._get_memory_info()
            if memory_info['is_high']:
                depth_extractor._handle_memory_pressure()
                stats.memory_cleanups += 1
        
        # Update progress with memory info
        current_memory = depth_extractor._get_memory_info()
        pbar.set_postfix({
            'Success': f"{stats.successful}/{stats.total_processed}",
            'Memory': f"{current_memory.get('usage_ratio', 0):.1%}"
        })
        pbar.update(1)

# Calculate final statistics
total_time = (datetime.now() - start_time).total_seconds()
stats.total_time = total_time
success_rate = (stats.successful / stats.total_processed) * 100 if stats.total_processed > 0 else 0

print(f"\n✅ Enhanced batch processing completed!")
print(f"\n📊 Enhanced Processing Results:")
print(f"   Total processed: {stats.total_processed}")
print(f"   Successful: {stats.successful}")
print(f"   Failed: {stats.failed}")
print(f"   Success rate: {success_rate:.1f}%")
print(f"   Memory cleanups: {stats.memory_cleanups}")
print(f"   Total time: {total_time:.1f}s")

if processing_times:
    print(f"\n⏱️ Performance Metrics:")
    print(f"   Average time: {np.mean(processing_times):.2f}s")
    print(f"   Fastest: {min(processing_times):.2f}s")
    print(f"   Slowest: {max(processing_times):.2f}s")

if quality_scores:
    print(f"\n🎯 Quality Metrics:")
    print(f"   Average quality: {np.mean(quality_scores):.3f}")
    print(f"   Quality range: {min(quality_scores):.3f} - {max(quality_scores):.3f}")

# Error summary
error_summary = depth_extractor.get_error_summary()
if error_summary['total_error_types'] > 0:
    print(f"\n❌ Error Summary:")
    for error_type, details in error_summary['error_breakdown'].items():
        print(f"   {error_type}: {details['count']} occurrences")

In [ ]:
# Save enhanced results with comprehensive metadata
print("💾 Saving enhanced results...")

enhanced_results = {
    'stage_info': {
        'stage_name': 'enhanced_depth_anything_v2_extraction',
        'timestamp': datetime.now().isoformat(),
        'model_used': selected_model,
        'device_used': device,
        'total_processing_time': total_time,
        'enhancement_features': [
            'adaptive_preprocessing',
            'memory_optimization',
            'error_recovery',
            'quality_assessment',
            'gpu_fallback'
        ]
    },
    'processing_statistics': {
        'total_images': stats.total_processed,
        'successful_extractions': stats.successful,
        'failed_extractions': stats.failed,
        'success_rate': success_rate,
        'memory_cleanups': stats.memory_cleanups,
        'gpu_fallbacks': stats.gpu_fallbacks
    },
    'performance_metrics': {
        'mean_processing_time': np.mean(processing_times) if processing_times else 0,
        'std_processing_time': np.std(processing_times) if processing_times else 0,
        'min_processing_time': min(processing_times) if processing_times else 0,
        'max_processing_time': max(processing_times) if processing_times else 0,
        'mean_quality_score': np.mean(quality_scores) if quality_scores else 0,
        'std_quality_score': np.std(quality_scores) if quality_scores else 0
    },
    'memory_analysis': {
        'peak_memory_usage': max([m.get('usage_ratio', 0) for m in memory_usage_history]) if memory_usage_history else 0,
        'average_memory_usage': np.mean([m.get('usage_ratio', 0) for m in memory_usage_history]) if memory_usage_history else 0,
        'memory_cleanup_frequency': stats.memory_cleanups / stats.total_processed if stats.total_processed > 0 else 0
    },
    'error_analysis': error_summary,
    'successful_results': successful_results,
    'failed_results': failed_results
}

try:
    save_stage_results(
        data=enhanced_results,
        stage_name="enhanced_depth_anything_v2_extraction",
        metadata={
            'model_used': selected_model,
            'processing_time': total_time,
            'success_rate': success_rate,
            'enhancement_level': 'advanced'
        }
    )
    print("✅ Enhanced results saved successfully!")
    
except Exception as e:
    print(f"⚠️ Warning: Failed to save results: {e}")

# Cleanup
depth_extractor.cleanup()
print("\n🧹 Cleanup completed")
print("✅ Enhanced depth extraction stage completed successfully!")